In [1]:
from sys import path
from os.path import dirname as dir

path.append("/Users/yaod_1/Desktop/webpage/text-simplification/ts-annoation-tool/data/analysis")

from utils.all import *

# 'batch_num' corresponds to the batches in the annotated folder
# 'preprocess' adds more data to each sentence, including SimpEval annotations
# The errors printed at the bottom are primarily because annotators will select a span
# but don't annotate that span
data = load_data('../annotated', batch_num=[5, 6], preprocess=True)

Loading files: ['../annotated/batch_5_anton.json', '../annotated/batch_5_ayush.json', '../annotated/batch_5_kelly.json', '../annotated/batch_5_rachel.json', '../annotated/batch_5_vinayak.json', '../annotated/batch_5_vishnesh.json', '../annotated/batch_6_anton.json', '../annotated/batch_6_ayush.json', '../annotated/batch_6_kelly.json', '../annotated/batch_6_rachel.json', '../annotated/batch_6_vinayak.json', '../annotated/batch_6_vishnesh.json']

Found users: {'vishnesh', 'anton', 'vinayak', 'kelly', 'ayush', 'rachel'}

anton - Batch 5, HIT 38 (ID 37) has 2 deletion edits but 1 annotations. Likely a missing annotation. Skipping edit type...
vinayak - Batch 6, HIT 16 (ID 45) has 1 insertion edits but -1 annotations. Likely a missing annotation. Skipping edit type...
rachel - Batch 5, HIT 21 (ID 65) has 4 deletion edits but 3 annotations. Likely a missing annotation. Skipping edit type...
rachel - Batch 5, HIT 21 (ID 65) has 4 insertion edits but 3 annotations. Likely a missing annotation.

In [19]:
data[0]["processed_annotations"]

[{'edit_type': 'deletion',
  'id': 0,
  'information_impact': <Information.LESS: 'Generalization'>,
  'type': <Quality.QUALITY: 'No Error'>,
  'family': <Family.CONTENT: 'Content'>,
  'grammar_error': False,
  'error_type': None,
  'rating': 3,
  'size': 0.008310249307479225,
  'reorder_level': None,
  'score': 0.0},
 {'edit_type': 'deletion',
  'id': 1,
  'information_impact': <Information.LESS: 'Generalization'>,
  'type': <Quality.QUALITY: 'No Error'>,
  'family': <Family.CONTENT: 'Content'>,
  'grammar_error': False,
  'error_type': None,
  'rating': 2,
  'size': 0.030470914127423823,
  'reorder_level': None,
  'score': 0.0},
 {'edit_type': 'deletion',
  'id': 2,
  'information_impact': <Information.LESS: 'Generalization'>,
  'type': <Quality.QUALITY: 'No Error'>,
  'family': <Family.CONTENT: 'Content'>,
  'grammar_error': False,
  'error_type': None,
  'rating': 3,
  'size': 0.06371191135734072,
  'reorder_level': None,
  'score': 0.0},
 {'edit_type': 'substitution',
  'id': 0,
  

In [14]:
Edit("Split")

<Edit.SPLIT: 'Split'>

In [20]:
!pwd

/Users/yaod_1/Desktop/webpage/text-simplification/ts-annoation-tool/data/edit_classification
